Random search over the parameters and hyperparameters of a bidirectional lstm, from this loop we extracted out best models scoring more than 0.7 that we ensembled at the end

In [ ]:
from google.colab import drive
drive.mount("./gdrive")

Mounted at ./gdrive


# Load Data

In [ ]:
DRIVE = "./gdrive/MyDrive/ann_dataset/HW2"

In [ ]:
import pickle
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
from keras.layers import Dense, Flatten,Dropout
from tensorflow import keras
import tensorflow as tf
import pandas as pd
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# load data split
with open(f"{DRIVE}/dataUsed/train_test_obj_oversampledkeep9", "rb") as f:
  x_trainO, x_testO, y_trainO, y_testO = pickle.load(f)

In [ ]:
# load data split
with open(f"{DRIVE}/dataUsed/train_test_objKeep9", "rb") as f:
  x_train, x_test, y_train, y_test = pickle.load(f)

In [ ]:
# load scaler
with open(f"{DRIVE}/dataUsed/scaler.p", "rb") as f:
    scaler = pickle.load(f)

In [ ]:
x_train_norm, y_train_norm = apply_scaler_to_new_data(x_train, y_train, scaler)
x_test_norm, y_test_norm = apply_scaler_to_new_data(x_test, y_test, scaler)
x_train_normO, y_train_normO= apply_scaler_to_new_data(x_trainO, y_trainO, scaler)
x_test_normO, y_test_normO = apply_scaler_to_new_data(x_testO, y_testO, scaler)

In [ ]:
np.mean(x_train_norm), np.max(x_train_norm), np.min(x_train_norm) # explore if normalization happened

(1.8948982610661986, 4043.1959847507133, -1967.3664388330392)

In [ ]:
np.mean(x_test_norm), np.max(x_test_norm), np.min(x_train_norm) # explore if normalization happened

(2.4541342982216214, 5816.222747418829, -1967.3664388330392)

# Functions

In [ ]:
def from_raw_to_dataframe(x_raw, y_raw):
    tabular_list = []
    for i_sample, x in enumerate(x_raw):
        for i_time, xx in enumerate(x):
            tabular_list.append([i_sample, i_time] + list(xx) + [y_raw[i_sample]])
    df_res = pd.DataFrame(tabular_list).rename(columns={
        0: 'sample_id',
        1: 'time',
        2: 'feat_1',
        3: 'feat_2',
        4: 'feat_3',
        5: 'feat_4',
        6: 'feat_5',
        7: 'feat_6',
        8: 'label'}, inplace=False, errors='raise')
    return df_res

In [ ]:
def build_sequences(df):
    # Sanity check to avoid runtime errors
    #assert window % stride == 0
    dataset = []
    labels = []
    for id in df['sample_id'].unique():
        # Take only meaningful features
        temp = df[df['sample_id'] == id][['feat_1','feat_2','feat_3','feat_4','feat_5','feat_6']].values
        # Save the label
        label = df[df['sample_id'] == id]['label'].values[0]
        labels.append(label)
        dataset.append(temp)

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [ ]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler


def normalize_data_feature_wise(x_original, y_original, max_value=1, quantile=False):
    x_original_tabular = from_raw_to_dataframe(x_original, y_original)
    if quantile:
        scaler = RobustScaler()
    else:
        scaler = MinMaxScaler(feature_range=(0, max_value))
    scaler.fit_transform(x_original_tabular[['feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6']])
    x_original_tabular[['feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6']] = scaler.transform(
        x_original_tabular[['feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6']])

    agg_x, agg_y = build_sequences(x_original_tabular)
    return agg_x, agg_y, scaler

In [ ]:
def apply_scaler_to_new_data(x,y,scaler):
  x_original_tabular = from_raw_to_dataframe(x, y)
  x_original_tabular[['feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6']] = scaler.transform(x_original_tabular[['feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6']])
  agg_x, agg_y = build_sequences(x_original_tabular)
  return agg_x, agg_y

In [ ]:
def shiftLeft(lst) :
    return lst[1:] + lst[:1]

def shiftRight(lst) :
    return lst[-1:] + lst[:-1]

def array_1D_to_2D(array, option=2):
  """ mappind 1D arrays to 2D images like arrays with the specified option"""
  base = list(array.copy())
  twoD_array = []

  if option == 0:
  # every row do a right shift
      for i in range(len(base)):
          twoD_array.append(base)
          base = shiftLeft(base)
  elif option == 1:
  # for iteration i do i right shifts of previous row
      for i in range(len(base)):
          twoD_array.append(base)
          for _ in range(i):
              base = shiftLeft(base)
  elif option == 2:
  # shift the same number of filter size (3)
      for i in range(len(base)):
          twoD_array.append(base)
          for _ in range(3):
              base = shiftLeft(base)
  return twoD_array

In [ ]:
def original_data_to_image_data(x, y, option):
    x_image = []
    y_image = []

    for i, label in enumerate(y):
        images_block = []
        for feat_id in range(x[i].shape[1]):
            ts = np.array(x[i][:, feat_id]) #extract time series
            image = array_1D_to_2D(ts, option=option)
            images_block.append(image)

        x_image.append(np.array(images_block))
        y_image.append(np.array(label))
        # reshape to use second option of accessing
        # x_original_image[0][0, :]
        # x_original_image_r[0][:,:, 0]
    x_image_r = np.reshape(x_image, (len(x), 36, 36, 6))
    return x_image_r, y_image

In [ ]:
# wrapper function that return a model composed by  pretrained model + custom FFNN as classifier at the end
def bottom_model_builder_classifier(
    base_model,
    dropout_rate=0.3,
    size_dense =1000
    ):
  l2_lambda = 1e-2
  inputs=tfk.Input(shape=(36, 36, 3))
  x = base_model(inputs)
  x = Flatten()(x)
  x = Dense(size_dense, activation='swish', kernel_initializer = tfk.initializers.HeUniform())(x)
  x = Dropout(dropout_rate, name="top_dropout_2")(x)
  x = Dense(256, activation='swish', kernel_initializer = tfk.initializers.HeUniform())(x)
  x = Dropout(dropout_rate, name="top_dropout_3")(x)
  outputs = tfkl.Dense(
      12, 
      activation='softmax',
      kernel_initializer = tfk.initializers.GlorotUniform())(x)


  # Connect input and output through the Model class
  tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='Densemodel')

  return tl_model

In [ ]:
def bottom_model_builder_fully_conv(
        base_model,
        dropout_rate,
):

    inputs=tfk.Input(shape=(36, 36, 3))
    x = base_model(inputs)
    x = tfkl.GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate, name="top_dropout")(x)
    outputs = tfkl.Dense(
        12,
        activation='softmax',
        kernel_initializer = tfk.initializers.GlorotUniform())(x)


    # Connect input and output through the Model class
    tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='FullConvmodel')

    return tl_model

In [ ]:
def make_model_conv(input_shape):
    input_layer = keras.layers.Input(input_shape)
    l2 = 1e-3
    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same", kernel_regularizer=tfk.regularizers.L2(l2))(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same", kernel_regularizer=tfk.regularizers.L2(l2))(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same", kernel_regularizer=tfk.regularizers.L2(l2))(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(12, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)

In [ ]:
def build_LSTM_classifier(classes=12):
    input_shape=x_train.shape[1:]
    seed=0
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # Feature extractor
    lstm = tfkl.LSTM(12, return_sequences=True)(input_layer) #Number of neurons means dimension of the cell
    #Return sequences is a parameter that decides to return each timestep output or only the final one (False)
    lstm = tfkl.LSTM(12)(lstm)
    dropout = tfkl.Dropout(0.3, seed=seed)(lstm)

    # Classifier
    classifier = tfkl.Dense(128, activation='relu')(dropout)
    output_layer = tfkl.Dense(classes, activation='softmax')(classifier)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    # model.compile(loss=tfk.losses.SparseCategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')
    model.compile(
      optimizer="adam",
      loss="sparse_categorical_crossentropy",
      metrics=["sparse_categorical_accuracy"],
)
    # Return the model
    return model

In [ ]:
from tensorflow.keras import regularizers

def build_LSTM_classifier(input_shape):
    REGULARIZING_FACTOR =1e-2
    DROPOUT = 0.4
    N = 56
    B = 2
    input_layer = keras.layers.Input(input_shape)
    x = tfkl.LSTM(N, return_sequences=True)(input_layer)
    x = keras.layers.Dropout(DROPOUT)(x)
    for i in range(B):
      x = tfkl.LSTM(N, return_sequences=True)(x)
      x = keras.layers.Dropout(DROPOUT)(x)
    x = tfkl.LSTM(N)(x)
    # x = keras.layers.Dense(12, kernel_regularizer=regularizers.l2(REGULARIZING_FACTOR), activation="relu")(x)
    x = keras.layers.Dropout(DROPOUT)(x)
    # x = keras.layers.Dense(12, activation="relu")(x)
    # x = keras.layers.Dropout(DROPOUT)(x)
    # x = keras.layers.Dense(6, activation="relu")(x)
    # x = keras.layers.Dropout(DROPOUT)(x)
    
    output_layer = keras.layers.Dense(12, activation="softmax")(x)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)

In [ ]:
from tensorflow.keras import regularizers

def bidirectional_LSTM(input_shape, DROPOUT, N , USE_DENSE, USE_BATCH_NORM, L2, B):

    input_layer = keras.layers.Input(input_shape)
    x = input_layer
    x = keras.layers.Bidirectional(keras.layers.LSTM(N, return_sequences=True))(x)
    if USE_BATCH_NORM:
        x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(DROPOUT)(x)
    for i in range(B):
      x = keras.layers.Bidirectional(keras.layers.LSTM(N, return_sequences=True))(x)
      x = keras.layers.Dropout(DROPOUT)(x)
    x = keras.layers.Bidirectional(keras.layers.LSTM(N))(x)

    x = keras.layers.Dropout(DROPOUT)(x)
    if USE_DENSE:
        x = keras.layers.Dense(6, activation="relu",kernel_regularizer=tf.keras.regularizers.L2(L2))(x)
        x = keras.layers.Dropout(DROPOUT)(x)
    output_layer = keras.layers.Dense(12, activation="softmax")(x)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)

# Random search for biLSTM params and hparams

In [ ]:
# callback list
callbacks_list = []
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=50, restore_best_weights=True)
callbacks_list.append(es_callback)

In [ ]:
log = []
for i in range(100):
    N = np.random.randint(28, 120)
    BATCH_SIZE = np.random.choice([8,32,64,128])
    SAVE_ABOVE = 0.71
    DROPOUT = np.random.uniform(0.0, 0.5)
    B = np.random.randint(0, 3)
    L2=np.random.uniform(0.0001, 0.015)
    USE_DENSE = np.random.choice([True, False], p=[0.2, 0.8])
    USE_AUGMENTED = np.random.choice([True, False], p=[0.2, 0.8])
    USE_BATCH_NORM = np.random.choice([True, False], p=[0.2, 0.8])
    DENSE_SIZE = np.random.randint(6, 64)
    LRATE = np.random.uniform(2e-3, 9e-2)

    model = bidirectional_LSTM(input_shape=x_train.shape[1:],
                               DROPOUT = DROPOUT,
                               N = N,
                               B = B,
                               L2=L2,
                               USE_DENSE=USE_DENSE,
                               USE_BATCH_NORM=USE_BATCH_NORM)
    if USE_AUGMENTED:
        train_dataset = tf.data.Dataset.from_tensor_slices((x_train_normO, y_train_normO)).batch(BATCH_SIZE)
        test_dataset = tf.data.Dataset.from_tensor_slices((x_test_normO, y_test_normO)).batch(BATCH_SIZE)
    else:
        train_dataset = tf.data.Dataset.from_tensor_slices((x_train_norm, y_train_norm)).batch(BATCH_SIZE)
        test_dataset = tf.data.Dataset.from_tensor_slices((x_test_norm, y_test_norm)).batch(BATCH_SIZE)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(LRATE),
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"],
    )
    history = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=500,
        callbacks=callbacks_list).history

    max_val_acc = np.max(model.history.history['val_sparse_categorical_accuracy'])
    if max_val_acc >= SAVE_ABOVE:
        name = f"{DRIVE}/exp/model{max_val_acc}"
        model.save(name)

    log.append({
        'val_accuracy': max_val_acc,
        'N':  N,
        'USE_BATCH_NORM':USE_BATCH_NORM,
        'BATCH_SIZE': BATCH_SIZE,
        'SAVE_ABOVE': SAVE_ABOVE,
        'DROPOUT': DROPOUT,
        'B': B,
        'L2': L2,
        'USE_DENSE': USE_DENSE,
        'DENSE_SIZE': DENSE_SIZE,
        'LRATE': LRATE,
        'USE_AUGMENTED': USE_AUGMENTED
    })

    with open(f"{DRIVE}/exp/scaler.p", "wb") as f:
        pickle.dump(log, f)

Epoch 1/500
16/16 [==============================] - 9s 140ms/step - loss: 2.3738 - sparse_categorical_accuracy: 0.3042 - val_loss: 1.9439 - val_sparse_categorical_accuracy: 0.3580
Epoch 2/500
16/16 [==============================] - 0s 23ms/step - loss: 1.9021 - sparse_categorical_accuracy: 0.3736 - val_loss: 1.7225 - val_sparse_categorical_accuracy: 0.4053
Epoch 3/500
16/16 [==============================] - 0s 20ms/step - loss: 1.7988 - sparse_categorical_accuracy: 0.4050 - val_loss: 2.0646 - val_sparse_categorical_accuracy: 0.3107
Epoch 4/500
16/16 [==============================] - 0s 21ms/step - loss: 1.7970 - sparse_categorical_accuracy: 0.3989 - val_loss: 1.8963 - val_sparse_categorical_accuracy: 0.3580
Epoch 5/500
16/16 [==============================] - 0s 21ms/step - loss: 1.7325 - sparse_categorical_accuracy: 0.4020 - val_loss: 1.9246 - val_sparse_categorical_accuracy: 0.3560
Epoch 6/500
16/16 [==============================] - 0s 20ms/step - loss: 1.7790 - sparse_categoric

KeyboardInterrupt: ignored

# Evaluation

In [ ]:
def show_confusion(model, validation_dataset):
  Y_pred = model.predict(validation_dataset)
  y_pred = np.argmax(Y_pred, axis=1)
  y_test= np.concatenate([y for x, y in validation_dataset], axis=0)

  cm = confusion_matrix(y_test, y_pred)

  disp = ConfusionMatrixDisplay(confusion_matrix=cm)

  disp.plot(cmap=plt.cm.Blues)
  plt.show()

  f1_scores = f1_score(y_test, y_pred, average=None)
  mean_f1 = np.mean(f1_scores)

  print(f"\nF1 scores: {f1_scores}")
  print(f"\nMean F1 : {mean_f1}")
  print(f"\nAccuracy : {accuracy_score(y_test, y_pred)}")

In [ ]:
show_confusion(model, test_dataset)

In [ ]:
metric = "sparse_categorical_accuracy"
plt.figure()
plt.plot(history[metric])
plt.plot(history["val_" + metric])
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.show()
plt.close()

In [ ]:
metric = "loss"
plt.figure()
plt.plot(history[metric])
plt.plot(history["val_" + metric])
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.show()
plt.close()